### Postgres SQL

In [1]:
import psycopg2
from datetime import datetime

class PGDatabase:
    def __init__(self, 
                 dbname, user, password,
                 host="localhost", port=5432, 
                 ):
        self.conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        self.cur = self.conn.cursor()

    def create_field_table(self):
        self.cur.execute("""
                    CREATE TABLE field (
                        field_id        VARCHAR,
                        field_name      VARCHAR NOT NULL,
                        unit            VARCHAR,
                        location        VARCHAR,
                        field_type      VARCHAR(10) NOT NULL CHECK (field_type IN ('OIL_PROD', 'GAS_PROD', 'OIL_PLAN', 'GAS_PLAN')),
                        conversion_factor FLOAT,
                        PRIMARY KEY (field_id, field_type)
                    );
                    """
            )
        self.conn.commit()

    def create_plan_prod_table(self):
        self.cur.execute("""
                CREATE TABLE plan_prod (
                    field_id    VARCHAR,
                    report_date DATE NOT NULL,
                    plan_type   VARCHAR(20) NOT NULL,
                    prod_ton    FLOAT,
                    prod_bbls   FLOAT,
                    prod_m3     FLOAT,
                    prod_ft3    FLOAT,
                    PRIMARY KEY (field_id, report_date, plan_type)
                );
                """
            )
        self.conn.commit()

    def create_daily_prod_table(self):
        self.cur.execute("""
                CREATE TABLE daily_prod (
                    field_id    VARCHAR,
                    report_date DATE NOT NULL,
                    prod_type   VARCHAR,
                    prod_ton    FLOAT,
                    prod_bbls   FLOAT,
                    prod_m3     FLOAT,
                    prod_ft3    FLOAT,
                    PRIMARY KEY (field_id, report_date, prod_type)
                );
                """
            )
        self.conn.commit()

    def get_latest_date_by_field(self, field_id, prod_type, query_date = '2025/07/01'): #"%Y/%m/%d"
        # Check latest date of data before the query_date, if no data, choose the closest data had data
        query_date = datetime.strptime(query_date, "%Y/%m/%d").date()
        found = False
        excepted_dates = []
        self.cur.execute("""
                SELECT report_date
                FROM daily_prod
                WHERE field_id = %s AND prod_type = %s AND report_date <= %s
                ORDER BY report_date DESC
                LIMIT 1;
            """, (field_id, prod_type, query_date))
        try:
            _last_date = self.cur.fetchone()[0]
        except:
            _last_date = None

        while not found:
            # Check the data by that date
            if _last_date is not None:
                self.cur.execute("""
                    SELECT * FROM daily_prod
                    WHERE field_id = %s AND report_date = %s AND prod_type = %s;
                """, (field_id, _last_date, prod_type))
                data = self.cur.fetchone()
                _prod_ton = data[3] if data is not None else None
                _prod_bbls = data[4] if data is not None else None
                _prod_m3 = data[5] if data is not None else None
                _prod_ft3 = data[6] if data is not None else None
                if prod_type=='OIL_PROD' and (_prod_ton is not None and _prod_bbls is not None):
                    found = True
                    return _last_date#.strftime("%Y/%m/%d")
                elif prod_type=='GAS_PROD' and (_prod_m3 is not None and _prod_ft3 is not None):
                    found = True
                    # Return data as string #"%Y/%m/%d"
                    return _last_date#.strftime("%Y/%m/%d")
                else:
                    excepted_dates.append(_last_date)
                    print(f"Daily data on {_last_date} is incomplete for {field_id} - {prod_type}. Trying previous date...")
                    # Temporarily remove that date and check again
                    self.cur.execute("""
                        SELECT report_date
                        FROM daily_prod
                        WHERE field_id = %s AND prod_type = %s AND report_date <= %s
                        AND report_date NOT IN %s
                        ORDER BY report_date DESC
                        LIMIT 1;
                    """, (field_id, prod_type, query_date, tuple(excepted_dates)))
                    try:
                        _last_date = self.cur.fetchone()[0]
                    except:
                        _last_date = None
            else:
                print(f"No data found for {field_id} - {prod_type}.")
                return None

    def get_all_table_names(self):
        self.cur.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public';
        """)
        return [row[0] for row in self.cur.fetchall()]
    
    def get_table(self, table_name):
        self.cur.execute(f"SELECT * FROM {table_name};")
        return self.cur.fetchall()

    def get_table_info(self, table_name):
        self.cur.execute(f"""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = '{table_name}';
        """)
        return self.cur.fetchall()

    def delete_table(self, table_name):
        self.cur.execute(f"DROP TABLE IF EXISTS {table_name};")
        self.conn.commit()
        print(f"Table {table_name} deleted.")

    def insert_field(self, field_id, field_name, unit, location, field_type, conversion_factor):
        self.cur.execute("""
            INSERT INTO field (field_id, field_name, unit, location, field_type, conversion_factor)
            VALUES (%s, %s, %s, %s, %s, %s);
        """, (field_id, field_name, unit, location, field_type, conversion_factor))
        self.conn.commit()

    def insert_plan_prod(self, field_id, report_date, plan_type, prod_ton, prod_bbls, prod_m3, prod_ft3):
        self.cur.execute("""
            INSERT INTO plan_prod (field_id, report_date, plan_type, prod_ton, prod_bbls, prod_m3, prod_ft3)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (field_id, report_date, plan_type, prod_ton, prod_bbls, prod_m3, prod_ft3))
        self.conn.commit()

    def insert_daily_prod(self, field_id, report_date, prod_type, prod_ton, prod_bbls, prod_m3, prod_ft3):
        self.cur.execute("""
            INSERT INTO daily_prod (field_id, report_date, prod_type, prod_ton, prod_bbls, prod_m3, prod_ft3)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (field_id, report_date, prod_type, prod_ton, prod_bbls, prod_m3, prod_ft3))
        self.conn.commit()

    def get_daily_prod_by_date(self, field_id, report_date, prod_type, unit='prod_bbls'):
        self.cur.execute(f"""
            SELECT {unit} FROM daily_prod
            WHERE field_id = %s AND report_date = %s AND prod_type = %s;
        """, (field_id, report_date, prod_type))
        return self.cur.fetchone()
    
    def get_conversion_factor(self, field_id, prod_type):
        # Get factor from field table by field_id and prod_type
        self.cur.execute("""
            SELECT conversion_factor FROM field
            WHERE field_id = %s AND field_type = %s;
        """, (field_id, prod_type))
        return self.cur.fetchone()[0]
    
    def get_field_name(self, field_id, field_type):
        # Get full name from field table by field_id and field_type
        self.cur.execute("""
            SELECT field_name FROM field
            WHERE field_id = %s AND field_type = %s;
        """, (field_id, field_type))
        return self.cur.fetchone()[0]

    #=======GET FOR REPORTING=========
    # Column C, D
    def get_accum_plan_year(self, field_id, year, plan_type):
        # Extract the accumulated production for a specific year by field_id
        self.cur.execute("""
            SELECT SUM(prod_ton) FROM plan_prod
            WHERE field_id = %s AND EXTRACT(YEAR FROM report_date) = %s AND plan_type = %s;
        """, (field_id, year, plan_type))
        return self.cur.fetchone()[0]
    
    # Column D
    def get_monthly_prod(self, field_id, month, prod_type, year=2025):
        # Extract the production for a specific month by field_id
        self.cur.execute("""
            SELECT SUM(prod_ton) FROM daily_prod
            WHERE field_id = %s AND EXTRACT(MONTH FROM report_date) = %s AND EXTRACT(YEAR FROM report_date) = %s AND prod_type = %s;
        """, (field_id, month, year, prod_type))
        return self.cur.fetchone()[0]

    def get_accum_daily(self, field_id, month, prod_type, year=2025):
        # Extract the accumulated production from Jan to a specific month by field_id
        if month < 1 or month > 12:
            print("Invalid month. Month must be between 1 and 12.")
            return None
        if month == 1:
            return 0
        accum_prod = 0
        for i in range(1, month):
            monthly_prod = self.get_monthly_prod(field_id, i, prod_type, year)
            if monthly_prod is not None:
                accum_prod += monthly_prod
        return accum_prod
    
    # Column H
    def get_monthly_plan_prod(self, field_id, month, plan_type, year=2025):
        # Extract the production for a specific month by field_id
        self.cur.execute("""
            SELECT SUM(prod_ton) FROM plan_prod
            WHERE field_id = %s AND EXTRACT(MONTH FROM report_date) = %s AND EXTRACT(YEAR FROM report_date) = %s AND plan_type = %s;
        """, (field_id, month, year, plan_type))
        return self.cur.fetchone()[0]
    
    # Column J
    def get_accum_monthly_prod_to_a_date(self, field_id, report_date, prod_type):
        # report_date in yyyy/mm/dd
        # Extract the accumulated production from 1/1 to the specified date by field_id
        self.cur.execute("""
            SELECT SUM(prod_ton) FROM daily_prod
            WHERE field_id = %s AND report_date BETWEEN %s AND %s AND prod_type = %s;
        """, (field_id, f'{report_date.year}-{report_date.month}-01', report_date, prod_type))
        return self.cur.fetchone()[0]
    
    # Column N
    def get_accum_daily_prod_up_to_date(self, field_id, report_date, prod_type, unit, year):
        # Extract accumulated prod up to date from 1/1/year
        self.cur.execute(f"""
            SELECT SUM({unit}) FROM daily_prod
            WHERE field_id = %s AND report_date BETWEEN '{year}-01-01' AND %s AND prod_type = %s;
        """, (field_id, report_date, prod_type))
        return self.cur.fetchone()[0]

In [2]:
POSTGRES_DB = "QLKTDB"
POSTGRES_USER = "dev"
POSTGRES_PASSWORD = "StR0&GP@sSW)R4"

PGDB = PGDatabase(
        dbname=POSTGRES_DB,
        user=POSTGRES_USER,
        password=POSTGRES_PASSWORD
    )

PGDB.get_all_table_names()

['field', 'plan_prod', 'daily_prod']

#### Query

In [ ]:
import pandas as pd
from datetime import datetime
def generate_report(query_date):
    query_date = datetime.strptime(query_date, "%Y/%m/%d")
    month = query_date.month
    year = query_date.year
    day = query_date.day
    # Column B
    field_names = [
        "Bạch Hổ & Rồng& 50%NR-ĐM",
        "NR-ĐM (Zarubezhneft)",
        "Cond. Dinh Cố & GPP Ca Mau",
        "Đại Hùng",
        "PM3-CAA",
        "46 CN",
        "Rạng Đông+Phương Đông",
        "Ruby+Pearl+Topaz+ Diamond",
        "STĐ+STV+STT+STN",
        "Cá Ngừ Vàng",
        "Tê  Giác Trắng",
        "Chim Sáo+ Dừa",
        "Lan Tây + Lan Đỏ",
        "Rồng Đôi+Rồng Đôi Tây",
        "Hải Sư Trắng +Hải Sư Đen",
        "Thăng Long + Đông Đô",
        "Hải Thạch + Mộc Tinh",
        "Kình Ngư Trắng - Nam",
        "Cá Tầm",
        "Thiên Ưng",
        "Sao Vàng -Đại Nguyệt",
        "Nhenhesky (49%VN)",
        "Algeria",
    ]
    fields =['BHR', 'DM', 'DC', 'DH', 'PM3CA', '46CN', 'RDPD', 'RPT', 'STD-STV-STT-STN', 'CNV', 'TGT', 'CS', 'LTLD', 'RD-RDT', 'HST-HSD', 'TLDD', 'HT-MT',  'KNT-N', 'CT', 'ThienUng', 'SVDN', 'Nhenhexky', 'Algeria']
    # Column C
    column_c = [PGDB.get_accum_plan_year(field_id=field, year=year, plan_type='KHSLCPGiaoOil') for field in fields]
    # Column D
    column_d = [PGDB.get_accum_plan_year(field_id=field, year=year, plan_type='KHQTOIL') for field in fields]
    # Column E
    sub_field_ids = [('BH', 'R', 'GT', 'ThT', 'NR'), 
                    'DM', 'DC-GPP', 'DH', 'PM3CAA', '46CN',
                    ('RangDong', 'PhuongDong'),
                    ('Ruby', 'Pearl', 'Topaz', 'Diamond'),
                    ('STD', 'STV', 'STD-DB', 'STT', 'STN'), 
                    'CNV', 'TGT', 'CS', 'LT', 'RD-RDT',
                    ('HST', 'HSD'), 'TLDD', 'HT-MT', 'KNT-N', 'CT', 
                    'ThienUng', 'SV', 'Nhenhexky', 'Algeria'
                    ]
    column_e = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _sub_field_prod = []
            for sub_field in _field:
                _prod = PGDB.get_accum_daily(field_id=sub_field, month=month, prod_type='OIL_PROD')/1000
                if _prod is not None:
                    _sub_field_prod.append(_prod)
                else:
                    _sub_field_prod.append(0)
            column_e.append(sum(_sub_field_prod))
        else:
            _prod = PGDB.get_accum_daily(field_id=_field, month=month, prod_type='OIL_PROD')
            column_e.append(_prod/1000 if _prod is not None else 0)
    # Column F
    column_f = [e * 100 / (1000 * c) if c != 0 else 0 for e, c in zip(column_e, column_c)]
    # Column G
    column_g = [e * 100 / (1000 * d) if d != 0 else 0 for e, d in zip(column_e, column_d)]
    # Column H
    column_h = [PGDB.get_monthly_plan_prod(field_id=field, month=month, plan_type='KHSLCPGiaoOil')*1000 for field in fields]
    # Column I
    column_i = [PGDB.get_monthly_plan_prod(field_id=field, month=month, plan_type='KHQTOIL')*1000 for field in fields]
    # Column J
    column_j = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _sub_field_prod = []
            for sub_field in _field:
                try:
                    _prod = PGDB.get_accum_monthly_prod_to_a_date(field_id=sub_field, report_date=query_date, prod_type='OIL_PROD')
                    _sub_field_prod.append(_prod/1000)
                except Exception as e:
                    _sub_field_prod.append(0)
            column_j.append(sum(_sub_field_prod))
        else:
            _prod = PGDB.get_accum_monthly_prod_to_a_date(field_id=_field, report_date=query_date, prod_type='OIL_PROD')
            column_j.append(_prod/1000 if _prod is not None else 0)
    # Column K
    column_k = [j * 100 / h if h != 0 else 0 for j, h in zip(column_j, column_h)]

    # Column L
    column_l = [j * 100 / i if i != 0 else 0 for j, i in zip(column_j, column_i)]

    # Column M
    column_m = [e + j for e, j in zip(column_e, column_j)]

    # Column N
    column_n = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _unused_fields = ['Pearl', 'Topaz', 'Diamond', 'HSD']
            _v = 0
            for sub_field in _field:
                if sub_field in _unused_fields:
                    _v += 0
                else:
                    _prod = PGDB.get_accum_daily_prod_up_to_date(field_id=sub_field, report_date=query_date, prod_type='OIL_PROD', unit='prod_bbls', year=year)
                    if _prod is not None:
                        _v += _prod
            column_n.append(_v)
        else:
            _prod = PGDB.get_accum_daily_prod_up_to_date(field_id=_field, report_date=query_date, prod_type='OIL_PROD', unit='prod_bbls', year=year)
            if _prod is not None:
                column_n.append(_prod)
            else:
                column_n.append(0)

    # Column O
    column_o = [m/(1000*c) if c != 0 else 0 for m, c in zip(column_m, column_c)]

    # Column P
    column_p = [m/(1000*d) if d != 0 else 0 for m, d in zip(column_m, column_d)]

    # Column Q
    column_q = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _unused_fields = ['Pearl', 'Topaz', 'Diamond', 'HSD']
            _v = 0
            for sub_field in _field:
                if sub_field in _unused_fields:
                    _v +=0
                else:
                    _daily_prod = PGDB.get_daily_prod_by_date(field_id=sub_field, report_date=query_date, prod_type='OIL_PROD', unit='prod_ton')
                    if _daily_prod is not None:
                        _v += _daily_prod[0]
            column_q.append(_v)
        else:
            _daily_prod = PGDB.get_daily_prod_by_date(field_id=_field, report_date=query_date, prod_type='OIL_PROD', unit='prod_ton')
            if _daily_prod is not None:
                column_q.append(_daily_prod[0])
            else:
                column_q.append(0)

    # Column R
    column_r = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _unused_fields = ['Pearl', 'Topaz', 'Diamond', 'HSD']
            _v = 0
            for sub_field in _field:
                if sub_field in _unused_fields:
                    _v +=0
                else:
                    _daily_prod = PGDB.get_daily_prod_by_date(field_id=sub_field, report_date=query_date, prod_type='OIL_PROD', unit='prod_bbls')
                    if _daily_prod is not None:
                        _v += _daily_prod[0]
            column_r.append(_v)
        else:
            _daily_prod = PGDB.get_daily_prod_by_date(field_id=_field, report_date=query_date, prod_type='OIL_PROD', unit='prod_bbls')
            if _daily_prod is not None:
                column_r.append(_daily_prod[0])
            else:
                column_r.append(0)
    data = {
        "Mỏ": field_names,
        "KHCP  (tr.tấn)": [ '%.2f' % elem for elem in column_c ],
        "KHQT  (tr.tấn)": [ '%.2f' % elem for elem in column_d ],
        "Tháng trước - Cộng dồn (ng.tấn)": [ '%.2f' % elem for elem in column_e ],
        "Tháng trước - %KHCP": [ '%.2f' % elem for elem in column_f ],
        "Tháng trước - %KHQT": [ '%.2f' % elem for elem in column_g ],
        "Tháng này - KHCP (ng.tấn)": [ '%.2f' % elem for elem in column_h ],
        "Tháng này - KHQT (ng.tấn)": [ '%.2f' % elem for elem in column_i ],
        "Tháng này - Thực hiện (ng.tấn)": [ '%.2f' % elem for elem in column_j ],
        "Tháng này - %KHCP": [ '%.2f' % elem for elem in column_k ],
        "Tháng này - %KHQT": [ '%.2f' % elem for elem in column_l ],
        "SL hiện tại - Cộng dồn (ng.tấn)": [ '%.2f' % elem for elem in column_m ],
        "SL hiện tại - Cộng dồn (thùng)": [ '%.2f' % elem for elem in column_n ],
        "SL hiện tại - %KHCP": [ '%.2f' % elem for elem in column_o ],
        "SL hiện tại - %KHQT": [ '%.2f' % elem for elem in column_p ],
        "SL ngày (tấn)": [ '%.2f' % elem for elem in column_q ],
        "SL ngày (thùng)": [ '%.2f' % elem for elem in column_r ]
    }
    return pd.DataFrame(data)

query_date = '2025/05/02'
generate_report(query_date)


,Mỏ,KHCP (tr.tấn),KHQT (tr.tấn),Tháng trước - Cộng dồn (ng.tấn),Tháng trước - %KHCP,Tháng trước - %KHQT,Tháng này - KHCP (ng.tấn),Tháng này - KHQT (ng.tấn),Tháng này - Thực hiện (ng.tấn),Tháng này - %KHCP,Tháng này - %KHQT,SL hiện tại - Cộng dồn (ng.tấn),SL hiện tại - Cộng dồn (thùng),SL hiện tại - %KHCP,SL hiện tại - %KHQT,SL ngày (tấn),SL ngày (thùng)
0,Bạch Hổ & Rồng& 50%NR-ĐM,2.70,2.85,927.92,34.37,32.56,231.30,259.80,15.41,6.66,5.93,943.34,7036743.45,0.35,0.33,7849.00,58521.02
1,NR-ĐM (Zarubezhneft),0.04,0.04,14.54,36.45,34.61,3.23,3.42,0.23,7.11,6.73,14.78,108389.40,0.37,0.35,115.00,843.64
2,Cond. Dinh Cố & GPP Ca Mau,0.04,0.05,19.52,45.53,43.39,3.45,3.63,0.33,9.50,9.03,19.85,167360.79,0.46,0.44,164.00,1382.52
3,Đại Hùng,0.40,0.45,89.26,22.54,19.96,10.98,15.05,2.30,20.98,15.31,91.57,680810.00,0.23,0.20,1115.80,8296.00
4,PM3-CAA,0.38,0.44,141.38,36.82,32.28,33.77,38.64,2.15,6.37,5.57,143.53,1076451.00,0.37,0.33,1084.13,8131.00
5,46 CN,0.02,0.02,5.87,39.13,34.31,1.25,1.43,0.09,7.43,6.49,5.97,44748.00,0.40,0.35,49.87,374.00
6,Rạng Đông+Phương Đông,0.40,0.42,140.66,35.18,33.32,34.35,38.71,2.40,6.99,6.20,143.06,1085784.00,0.36,0.34,1199.74,9100.00
7,Ruby+Pearl+Topaz+ Diamond,0.29,0.33,121.22,41.18,36.62,25.43,32.83,2.04,8.01,6.20,123.25,916998.00,0.42,0.37,1019.76,7587.00
8,STĐ+STV+STT+STN,1.19,1.30,420.80,35.21,32.39,105.33,114.85,6.90,6.55,6.00,427.70,3282050.17,0.36,0.33,3448.28,26506.00
9,Cá Ngừ Vàng,0.13,0.13,43.50,33.99,33.19,11.34,11.73,0.73,6.43,6.21,44.23,345006.00,0.35,0.34,362.82,2830.00


### [OIL TABLE] Generate Table with given daily data

In [ ]:
# Check latest date has data by field in daily_prod
query_date = '2025/08/02' #"%Y/%m/%d"

def generate_oil_report_w_latest_data(query_date):
    sub_field_ids = [('BH', 'R', 'GT', 'ThT', 'NR'), 
                        'DM', 'DC-GPP', 'DH', 'PM3CAA', '46CN',
                        ('RangDong', 'PhuongDong'),
                        ('Ruby', 'Pearl', 'Topaz', 'Diamond'),
                        ('STD', 'STV', 'STD-DB', 'STT', 'STN'), 
                        'CNV', 'TGT', 'CS', 'LT', 'RD-RDT',
                        ('HST', 'HSD'), 'TLDD', 'HT-MT', 'KNT-N', 'CT', 
                        'ThienUng', 'SV', 'Nhenhexky', 'Algeria'
                    ]
    _latest_dates_by_field = {}
    for sub_field_id in sub_field_ids:
        if isinstance(sub_field_id, tuple):
            dates = []
            for field_id in sub_field_id:
                _date = PGDB.get_latest_date_by_field(field_id, 'OIL_PROD', query_date)
                dates.append(_date)
            valid_dates = [d for d in dates if d is not None]
            _date = max(valid_dates) if valid_dates else None
            _latest_dates_by_field[sub_field_id] = _date
        else:
            _date = PGDB.get_latest_date_by_field(sub_field_id, 'OIL_PROD', query_date)
            _latest_dates_by_field[sub_field_id] = _date

    report = generate_report(query_date)
    _reformat_date = datetime.strptime(query_date, "%Y/%m/%d")
    report['Dữ liệu cập nhật đến ngày'] = _reformat_date.strftime("%d/%m/%Y")
    for i, (k, v) in enumerate(_latest_dates_by_field.items()):
        if v is not None and v != datetime.strptime(query_date, "%Y/%m/%d").date():
            _query_date = v.strftime("%Y/%m/%d")
            print(f"Field {k} has latest data on {_query_date}, not {query_date}")
            _new_query_report = generate_report(_query_date)
            _reformat_date = datetime.strptime(_query_date, "%Y/%m/%d")
            _new_query_report['Dữ liệu cập nhật đến ngày'] = _reformat_date.strftime("%d/%m/%Y")
            report.at[i, 'Dữ liệu cập nhật đến ngày'] = v.strftime("%d/%m/%Y")
            report.loc[i, :] = _new_query_report.loc[i, :].values
        else:
            print(f"Field {k} has latest data on same date")
    return report
generate_oil_report_w_latest_data(query_date)

No data found for Pearl - OIL_PROD.
No data found for Topaz - OIL_PROD.
No data found for Diamond - OIL_PROD.
No data found for HSD - OIL_PROD.
No data found for KNT-N - OIL_PROD.
Field ('BH', 'R', 'GT', 'ThT', 'NR') has latest data on 2025/07/01, not 2025/08/02
Field DM has latest data on 2025/07/01, not 2025/08/02
Field DC-GPP has latest data on 2025/07/01, not 2025/08/02
Field DH has latest data on 2025/07/01, not 2025/08/02
Field PM3CAA has latest data on 2025/07/01, not 2025/08/02
Field 46CN has latest data on 2025/07/01, not 2025/08/02
Field ('RangDong', 'PhuongDong') has latest data on 2025/07/01, not 2025/08/02
Field ('Ruby', 'Pearl', 'Topaz', 'Diamond') has latest data on 2025/07/01, not 2025/08/02
Field ('STD', 'STV', 'STD-DB', 'STT', 'STN') has latest data on 2025/07/01, not 2025/08/02
Field CNV has latest data on 2025/07/01, not 2025/08/02
Field TGT has latest data on 2025/07/01, not 2025/08/02
Field CS has latest data on 2025/07/01, not 2025/08/02
Field LT has latest data 

,Mỏ,KHCP (tr.tấn),KHQT (tr.tấn),Tháng trước - Cộng dồn (ng.tấn),Tháng trước - %KHCP,Tháng trước - %KHQT,Tháng này - KHCP (ng.tấn),Tháng này - KHQT (ng.tấn),Tháng này - Thực hiện (ng.tấn),Tháng này - %KHCP,Tháng này - %KHQT,SL hiện tại - Cộng dồn (ng.tấn),SL hiện tại - Cộng dồn (thùng),SL hiện tại - %KHCP,SL hiện tại - %KHQT,SL ngày (tấn),SL ngày (thùng),Dữ liệu cập nhật đến ngày
0,Bạch Hổ & Rồng& 50%NR-ĐM,2.70,2.85,1406.08,52.08,49.34,233.80,241.48,7.99,3.42,3.31,1414.07,10546506.46,0.52,0.50,7992.00,59594.95,01/07/2025
1,NR-ĐM (Zarubezhneft),0.04,0.04,21.26,53.28,50.59,3.31,3.69,0.10,3.02,2.71,21.36,156696.96,0.54,0.51,100.00,733.60,01/07/2025
2,Cond. Dinh Cố & GPP Ca Mau,0.04,0.05,29.20,68.09,64.89,3.53,3.61,0.14,3.94,3.85,29.34,247319.34,0.68,0.65,139.00,1171.77,01/07/2025
3,Đại Hùng,0.40,0.45,197.97,49.99,44.28,41.89,44.50,2.21,5.27,4.96,200.17,1488293.00,0.51,0.45,2205.51,16398.00,01/07/2025
4,PM3-CAA,0.38,0.44,209.48,54.55,47.83,36.27,42.70,1.10,3.03,2.58,210.58,1579380.00,0.55,0.48,1100.40,8253.00,01/07/2025
5,46 CN,0.02,0.02,9.05,60.28,52.85,1.25,1.73,0.06,4.67,3.38,9.11,68300.00,0.61,0.53,58.40,438.00,01/07/2025
6,Rạng Đông+Phương Đông,0.40,0.42,212.86,53.23,50.42,30.97,31.41,1.23,3.96,3.91,214.09,1624862.02,0.54,0.51,1226.72,9310.00,01/07/2025
7,Ruby+Pearl+Topaz+ Diamond,0.29,0.33,181.16,61.54,54.73,25.79,25.79,0.97,3.76,3.76,182.13,1355014.00,0.62,0.55,969.09,7210.00,01/07/2025
8,STĐ+STV+STT+STN,1.19,1.30,637.87,53.38,49.10,100.13,111.22,3.46,3.45,3.11,641.33,4923572.18,0.54,0.49,3458.33,26609.86,01/07/2025
9,Cá Ngừ Vàng,0.13,0.13,65.34,51.04,49.85,10.13,10.24,0.39,3.87,3.83,65.73,512682.00,0.51,0.50,392.05,3058.00,01/07/2025


### GAS TABLE

In [7]:
import psycopg2

class PGDatabase:
    def __init__(self, 
                 dbname, user, password,
                 host="localhost", port=5432, 
                 ):
        self.conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        self.cur = self.conn.cursor()
    
    def get_all_table_names(self):
        self.cur.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public';
        """)
        return [row[0] for row in self.cur.fetchall()]
    
    def get_latest_date_by_field(self, field_id, prod_type='GAS_PROD', query_date = '2025/07/01'): #"%Y/%m/%d"
        # Check latest date of data before the query_date, if no data, choose the closest data had data
        query_date = datetime.strptime(query_date, "%Y/%m/%d").date()
        found = False
        excepted_dates = []
        self.cur.execute("""
                SELECT report_date
                FROM daily_prod
                WHERE field_id = %s AND prod_type = %s AND report_date <= %s
                ORDER BY report_date DESC
                LIMIT 1;
            """, (field_id, prod_type, query_date))
        try:
            _last_date = self.cur.fetchone()[0]
        except:
            _last_date = None

        while not found:
            # Check the data by that date
            if _last_date is not None:
                self.cur.execute("""
                    SELECT * FROM daily_prod
                    WHERE field_id = %s AND report_date = %s AND prod_type = %s;
                """, (field_id, _last_date, prod_type))
                data = self.cur.fetchone()
                _prod_ton = data[3] if data is not None else None
                _prod_bbls = data[4] if data is not None else None
                _prod_m3 = data[5] if data is not None else None
                _prod_ft3 = data[6] if data is not None else None
                if prod_type=='OIL_PROD' and (_prod_ton is not None and _prod_bbls is not None):
                    found = True
                    return _last_date#.strftime("%Y/%m/%d")
                elif prod_type=='GAS_PROD' and (_prod_m3 is not None and _prod_ft3 is not None):
                    found = True
                    # Return data as string #"%Y/%m/%d"
                    return _last_date#.strftime("%Y/%m/%d")
                else:
                    excepted_dates.append(_last_date)
                    print(f"Daily data on {_last_date} is incomplete for {field_id} - {prod_type}. Trying previous date...")
                    # Temporarily remove that date and check again
                    self.cur.execute("""
                        SELECT report_date
                        FROM daily_prod
                        WHERE field_id = %s AND prod_type = %s AND report_date <= %s
                        AND report_date NOT IN %s
                        ORDER BY report_date DESC
                        LIMIT 1;
                    """, (field_id, prod_type, query_date, tuple(excepted_dates)))
                    try:
                        _last_date = self.cur.fetchone()[0]
                    except:
                        _last_date = None
            else:
                print(f"No data found for {field_id} - {prod_type}.")
                return None
    
    #=======GET FOR REPORTING=========
    # Column C, D
    def get_accum_plan_year(self, field_id, year, plan_type):
        # Extract the accumulated production for a specific year by field_id
        self.cur.execute("""
            SELECT SUM(prod_m3) FROM plan_prod
            WHERE field_id = %s AND EXTRACT(YEAR FROM report_date) = %s AND plan_type = %s;
        """, (field_id, year, plan_type))
        return self.cur.fetchone()[0]
    
    def get_data_by_field(self, field_id):
        """Get datetime and prod_m3 by field_id and prod_type='GAS_PROD'"""
        self.cur.execute("""
            SELECT report_date, prod_m3 FROM daily_prod
            WHERE field_id = %s AND prod_type = 'GAS_PROD';
        """, (field_id,))
        return self.cur.fetchall()

    def get_all_production_data(self):
        """Get all production data for all fields and prod_type='GAS_PROD'"""
        self.cur.execute("""
            SELECT field_id, report_date, prod_m3 FROM daily_prod
            WHERE prod_type = 'GAS_PROD';
        """)
        return self.cur.fetchall()
    
    # Column D
    def get_monthly_prod(self, field_id, month, prod_type, year=2025):
        # Extract the production for a specific month by field_id
        self.cur.execute("""
            SELECT SUM(prod_m3) FROM daily_prod
            WHERE field_id = %s AND EXTRACT(MONTH FROM report_date) = %s AND EXTRACT(YEAR FROM report_date) = %s AND prod_type = %s;
        """, (field_id, month, year, prod_type))
        return self.cur.fetchone()[0]
    
    def get_accum_daily(self, field_id, month, prod_type, year=2025):
        # Extract the accumulated production from Jan to a specific month by field_id
        if month < 1 or month > 12:
            print("Invalid month. Month must be between 1 and 12.")
            return None
        if month == 1:
            return 0
        accum_prod = 0
        for i in range(1, month):
            monthly_prod = self.get_monthly_prod(field_id, i, prod_type, year)
            if monthly_prod is not None:
                accum_prod += monthly_prod
        return accum_prod
    
    # Column H
    def get_monthly_plan_prod(self, field_id, month, plan_type, year=2025):
        # Extract the production for a specific month by field_id
        self.cur.execute("""
            SELECT SUM(prod_m3) FROM plan_prod
            WHERE field_id = %s AND EXTRACT(MONTH FROM report_date) = %s AND EXTRACT(YEAR FROM report_date) = %s AND plan_type = %s;
        """, (field_id, month, year, plan_type))
        return self.cur.fetchone()[0]
    
    # Column J
    def get_accum_monthly_prod_to_a_date(self, field_id, report_date, prod_type):
        # report_date in yyyy/mm/dd
        # Extract the accumulated production from 1/1 to the specified date by field_id
        self.cur.execute("""
            SELECT SUM(prod_m3) FROM daily_prod
            WHERE field_id = %s AND report_date BETWEEN %s AND %s AND prod_type = %s;
        """, (field_id, f'{report_date.year}-{report_date.month}-01', report_date, prod_type))
        return self.cur.fetchone()[0]
    
    # Column N
    def get_accum_daily_prod_up_to_date(self, field_id, report_date, prod_type, unit, year):
        # Extract accumulated prod up to date from 1/1/year
        self.cur.execute(f"""
            SELECT SUM({unit}) FROM daily_prod
            WHERE field_id = %s AND report_date BETWEEN '{year}-01-01' AND %s AND prod_type = %s;
        """, (field_id, report_date, prod_type))
        return self.cur.fetchone()[0]
    
    # Column Q, R
    def get_daily_prod_by_date(self, field_id, report_date, prod_type, unit='prod_bbls'):
        self.cur.execute(f"""
            SELECT {unit} FROM daily_prod
            WHERE field_id = %s AND report_date = %s AND prod_type = %s;
        """, (field_id, report_date, prod_type))
        return self.cur.fetchone()


POSTGRES_DB = "QLKTDB"
POSTGRES_USER = "dev"
POSTGRES_PASSWORD = "StR0&GP@sSW)R4"

PGDB = PGDatabase(
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)
PGDB.get_all_table_names()

['field', 'plan_prod', 'daily_prod']

In [ ]:
import pandas as pd
from datetime import datetime

import pandas as pd
from datetime import datetime
def generate_gas_report(query_date):
    query_date = datetime.strptime(query_date, "%Y/%m/%d")
    month = query_date.month
    year = query_date.year
    day = query_date.day

    # Column B
    field_names = [
        'Bạch Hổ+ Rồng',
        'Tê Giác Trắng',
        'Rạng Đông+Phương Đông',
        'Chim Sáo+  Dừa',
        'STĐ+STV+STT+STN',
        'Cá Ngừ Vàng',
        'Kình Ngư Trắng',
        'Lan Tây+Lan Đỏ',
        'Rồng Đôi+Rồng Đôi Tây',
        'Lô PM3-CAA ( tổng khí về bờ)',
        'Hải Sư Trắng +Hải Sư Đen',
        'Hải Thạch + Mộc Tinh',
        'Thái Bình',
        'Thiên Ưng',
        'Sao Vàng -Đại Nguyet',
        'Đại Hùng',
        'Cá Tầm',
    ]
    KHQT_fields = ['BH', 'TGT', 'RangDong', 'CS', 'STD-STV-STT', 'CNV', 'KNT-N', 'LTLD', 'RD-RDT', 'PM3CA-46CN', 'HST-HSD', 'HT-MT', 'TB', 'ThienUng', 'SVDN', 'DH', 'CT']
    KHCP_fields = ['BH', 'TGT', 'RDPD', 'CS-D', 'STD-STV-STT-STN', 'CNV', 'KNT-N', 'LTLD', 'RD-RDT', 'PM3CA-46CN', 'HST-HSD', 'HT-MT', 'TB', 'ThienUng', 'SVDN', 'DH', 'CT']

    # Column C
    column_c = [PGDB.get_accum_plan_year(field_id=field, year=year, plan_type='KHSLCPGiaoGas') for field in KHCP_fields]
    # Column D
    column_d = [PGDB.get_accum_plan_year(field_id=field, year=year, plan_type='KHQTGAS') for field in KHQT_fields]

    # Column E
    sub_field_ids = [('BH', 'R'), 
                    'TGT',
                    ('RangDong', 'PhuongDong'),
                    'CS',
                    ('STD', 'STV', 'STD-DB', 'STT', 'STN'), 
                    'CNV', 'KNT-N', 'LT', 'RD-RDT', 
                    'PM3-46CN',
                    'HST-HSD', 'HT', 'ThaiBinh', 'ThienUng', 'SV', 'DH', 'CT'
                    ]
    column_e = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _sub_field_prod = []
            for sub_field in _field:
                _accum_prod = PGDB.get_accum_daily(field_id=sub_field, month=month, prod_type='GAS_PROD')
                if _accum_prod is not None:
                    _sub_field_prod.append(_accum_prod)
                else:
                    _sub_field_prod.append(0)
            column_e.append(sum(_sub_field_prod))
        else:
            _accum_prod = PGDB.get_accum_daily(field_id=_field, month=month, prod_type='GAS_PROD')
            if _accum_prod is not None:
                column_e.append(_accum_prod)
            else:
                column_e.append(0)

    # Column F
    column_f = [e * 100 / (c) if c != 0 else 0 for e, c in zip(column_e, column_c)]
    # Column G
    column_g = [e * 100 / (d) if d != 0 else 0 for e, d in zip(column_e, column_d)]
    # Column H
    column_h = [PGDB.get_monthly_plan_prod(field_id=field, month=month, plan_type='KHSLCPGiaoGas') for field in KHCP_fields]
    # Column I
    column_i = [PGDB.get_monthly_plan_prod(field_id=field, month=month, plan_type='KHQTGAS') for field in KHQT_fields]
    # Column J
    column_j = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _sub_field_prod = []
            for sub_field in _field:
                try:
                    _prod = PGDB.get_accum_monthly_prod_to_a_date(field_id=sub_field, report_date=query_date, prod_type='GAS_PROD')
                    if _prod is None:
                        _prod = 0
                    _sub_field_prod.append(_prod)
                except Exception as e:
                    _sub_field_prod.append(0)
            column_j.append(sum(_sub_field_prod))
        else:
            _prod = PGDB.get_accum_monthly_prod_to_a_date(field_id=_field, report_date=query_date, prod_type='GAS_PROD')
            if _prod is not None:
                column_j.append(_prod)
            else:
                column_j.append(0)
    # # Column K
    column_k = [j * 100 / h if h != 0 else 0 for j, h in zip(column_j, column_h)]

    # Column L
    column_l = [j * 100 / i if i != 0 else 0 for j, i in zip(column_j, column_i)]

    # Column M
    column_m = [e + j for e, j in zip(column_e, column_j)]

    # Column N
    column_n = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _unused_fields = ['Pearl', 'Topaz', 'Diamond', 'HSD']
            _v = 0
            for sub_field in _field:
                if sub_field in _unused_fields:
                    _v += 0
                else:
                    _prod = PGDB.get_accum_daily_prod_up_to_date(field_id=sub_field, report_date=query_date, prod_type='GAS_PROD', unit='prod_ft3', year=year)
                    if _prod is None:
                        _prod = 0
                    _v += _prod
            column_n.append(_v)
        else:
            _prod = PGDB.get_accum_daily_prod_up_to_date(field_id=_field, report_date=query_date, prod_type='GAS_PROD', unit='prod_ft3', year=year)
            if _prod is not None:
                column_n.append(_prod)
            else:
                column_n.append(0)

    # Column O
    column_o = [100*m/c if c != 0 else 0 for m, c in zip(column_m, column_c)]

    # Column P
    column_p = [100*m/d if d != 0 else 0 for m, d in zip(column_m, column_d)]

    # Column Q
    column_q = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _v = 0
            for sub_field in _field:
                _prod = PGDB.get_daily_prod_by_date(field_id=sub_field, report_date=query_date, prod_type='GAS_PROD', unit='prod_m3')
                if _prod is None:
                    _prod = 0
                    _v += _prod
                else:
                    _v += _prod[0]
            column_q.append(_v)
        else:
            _prod = PGDB.get_daily_prod_by_date(field_id=_field, report_date=query_date, prod_type='GAS_PROD', unit='prod_m3')
            if _prod is not None:
                column_q.append(_prod[0])
            else:
                column_q.append(0)

    # Column R
    column_r = []
    for _field in sub_field_ids:
        if isinstance(_field, tuple):
            _unused_fields = ['Pearl', 'Topaz', 'Diamond', 'HSD']
            _v = 0
            for sub_field in _field:
                if sub_field in _unused_fields:
                    _v +=0
                else:
                    _prod = PGDB.get_daily_prod_by_date(field_id=sub_field, report_date=query_date, prod_type='GAS_PROD', unit='prod_ft3')
                    if _prod is None:
                        _prod = 0
                        _v += _prod
                    else:
                        _v += _prod[0]
            column_r.append(_v)
        else:
            _prod = PGDB.get_daily_prod_by_date(field_id=_field, report_date=query_date, prod_type='GAS_PROD', unit='prod_ft3')
            if _prod is not None:
                column_r.append(_prod[0])
            else:
                column_r.append(0)

    data = {
            "Mỏ": field_names,
            "KHCP  (tr.m3)": ["%.2f" % elem for elem in column_c],
            "KHQT  (tr.m3)": ["%.2f" % elem for elem in column_d],
            "Tháng trước - Cộng dồn (tr.m3)": ["%.2f" % elem for elem in column_e],
            "Tháng trước - %KHCP": ["%.2f" % elem for elem in column_f],
            "Tháng trước - %KHQT": ["%.2f" % elem for elem in column_g],
            "Tháng này - KHCP (tr.m3)": ["%.2f" % elem for elem in column_h],
            "Tháng này - KHQT (tr.m3)": ["%.2f" % elem for elem in column_i],
            "Tháng này - Thực hiện (tr.m3)": ["%.2f" % elem for elem in column_j],
            "Tháng này - %KHCP": ["%.2f" % elem for elem in column_k],
            "Tháng này - %KHQT": ["%.2f" % elem for elem in column_l],
            "SL hiện tại - Cộng dồn (tr.m3)": ["%.2f" % elem for elem in column_m],
            "SL hiện tại - Cộng dồn (tr.ft3)": ["%.2f" % elem for elem in column_n],
            "SL hiện tại - %KHCP": ["%.2f" % elem for elem in column_o],
            "SL hiện tại - %KHQT": ["%.2f" % elem for elem in column_p],
            "SL ngày (tr.m3)": ["%.2f" % elem for elem in column_q],
            "SL ngày (tr.ft3)": ["%.2f" % elem for elem in column_r]
        }
    return pd.DataFrame(data)

query_date = '2025/05/02'
generate_gas_report(query_date)

,Mỏ,KHCP (tr.m3),KHQT (tr.m3),Tháng trước - Cộng dồn (tr.m3),Tháng trước - %KHCP,Tháng trước - %KHQT,Tháng này - KHCP (tr.m3),Tháng này - KHQT (tr.m3),Tháng này - Thực hiện (tr.m3),Tháng này - %KHCP,Tháng này - %KHQT,SL hiện tại - Cộng dồn (tr.m3),SL hiện tại - Cộng dồn (tr.ft3),SL hiện tại - %KHCP,SL hiện tại - %KHQT,SL ngày (tr.m3),SL ngày (tr.ft3)
0,Bạch Hổ+ Rồng,90.00,122.00,38.90,43.22,31.88,10.21,14.18,0.46,4.46,3.21,39.35,1389.77,43.73,32.26,0.24,8.37
1,Tê Giác Trắng,34.53,34.53,21.26,61.58,61.58,2.98,2.37,0.39,13.23,16.65,21.66,764.82,62.72,62.72,0.20,6.97
2,Rạng Đông+Phương Đông,34.35,100.00,21.59,62.87,21.59,2.09,5.69,0.59,28.23,10.39,22.18,783.36,64.59,22.18,0.27,9.67
3,Chim Sáo+ Dừa,113.83,117.30,28.77,25.27,24.53,9.34,10.59,0.56,5.97,5.27,29.33,1035.63,25.76,25.00,0.27,9.71
4,STĐ+STV+STT+STN,405.20,441.19,107.88,26.63,24.45,34.41,41.89,1.22,3.54,2.91,109.10,3852.87,26.93,24.73,0.48,17.00
5,Cá Ngừ Vàng,27.80,30.00,18.43,66.30,61.44,2.50,2.23,0.53,21.05,23.60,18.96,669.46,68.19,63.19,0.26,9.12
6,Lan Tây+Lan Đỏ,0.00,30.81,41.29,0.00,134.01,0.00,0.00,0.34,0.00,0.00,41.63,1470.29,0.00,135.13,0.27,9.46
7,Rồng Đôi+Rồng Đôi Tây,261.20,266.60,118.52,45.38,44.46,24.51,23.63,1.06,4.33,4.49,119.59,4223.12,45.78,44.86,0.81,28.46
8,Lô PM3-CAA ( tổng khí về bờ),992.00,1935.42,621.19,62.62,32.10,81.96,162.65,10.68,13.03,6.57,631.88,22314.49,63.70,32.65,5.36,189.39
9,Hải Sư Trắng +Hải Sư Đen,23.00,38.00,3.69,16.02,9.70,2.20,5.04,0.15,6.69,2.92,3.83,135.34,16.66,10.09,0.07,2.60


### [GAS TABLE] Generate Table with given daily data

In [ ]:
# Check latest date has data by field in daily_prod
query_date = '2025/08/02' #"%Y/%m/%d"
def generate_gas_report_w_latest_data(query_date):
    sub_field_ids = [('BH', 'R'), 
                    'TGT',
                    ('RangDong', 'PhuongDong'),
                    'CS',
                    ('STD', 'STV', 'STD-DB', 'STT', 'STN'), 
                    'CNV', 'KNT-N', 'LT', 'RD-RDT', 
                    'PM3-46CN',
                    'HST-HSD', 'HT', 'ThaiBinh', 'ThienUng', 'SV', 'DH', 'CT'
                    ]
    _latest_dates_by_field = {}
    for sub_field_id in sub_field_ids:
        if isinstance(sub_field_id, tuple):
            dates = []
            for field_id in sub_field_id:
                _date = PGDB.get_latest_date_by_field(field_id, 'GAS_PROD', query_date)
                dates.append(_date)
            valid_dates = [d for d in dates if d is not None]
            _date = max(valid_dates) if valid_dates else None
            _latest_dates_by_field[sub_field_id] = _date
        else:
            _date = PGDB.get_latest_date_by_field(sub_field_id, 'GAS_PROD', query_date)
            _latest_dates_by_field[sub_field_id] = _date

    report = generate_gas_report(query_date)
    _reformat_date = datetime.strptime(query_date, "%Y/%m/%d")
    report['Dữ liệu cập nhật đến ngày'] = _reformat_date.strftime("%d/%m/%Y")
    for i, (k, v) in enumerate(_latest_dates_by_field.items()):
        if v is not None and v != datetime.strptime(query_date, "%Y/%m/%d").date():
            _query_date = v.strftime("%Y/%m/%d")
            print(f"Field {k} has latest data on {_query_date}, not {query_date}")
            _new_query_report = generate_gas_report(_query_date)
            _reformat_date = datetime.strptime(_query_date, "%Y/%m/%d")
            _new_query_report['Dữ liệu cập nhật đến ngày'] = _reformat_date.strftime("%d/%m/%Y")
            report.at[i, 'Dữ liệu cập nhật đến ngày'] = v.strftime("%d/%m/%Y")
            report.loc[i, :] = _new_query_report.loc[i, :].values
        else:
            print(f"Field {k} has latest data on same date")
    return report

No data found for R - GAS_PROD.
No data found for PhuongDong - GAS_PROD.
No data found for STD - GAS_PROD.
No data found for STV - GAS_PROD.
No data found for STD-DB - GAS_PROD.
No data found for STN - GAS_PROD.
Field ('BH', 'R') has latest data on 2025/07/01, not 2025/08/02
Field TGT has latest data on 2025/07/01, not 2025/08/02
Field ('RangDong', 'PhuongDong') has latest data on 2025/07/01, not 2025/08/02
Field CS has latest data on 2025/07/01, not 2025/08/02
Field ('STD', 'STV', 'STD-DB', 'STT', 'STN') has latest data on 2025/07/01, not 2025/08/02
Field CNV has latest data on 2025/07/01, not 2025/08/02
Field LT has latest data on 2025/07/01, not 2025/08/02
Field RD-RDT has latest data on 2025/07/01, not 2025/08/02
Field PM3-46CN has latest data on 2025/07/01, not 2025/08/02
Field HST-HSD has latest data on 2025/07/01, not 2025/08/02
Field HT has latest data on 2025/07/01, not 2025/08/02
Field ThaiBinh has latest data on 2025/07/01, not 2025/08/02
Field ThienUng has latest data on 20

,Mỏ,KHCP (tr.m3),KHQT (tr.m3),Tháng trước - Cộng dồn (tr.m3),Tháng trước - %KHCP,Tháng trước - %KHQT,Tháng này - KHCP (tr.m3),Tháng này - KHQT (tr.m3),Tháng này - Thực hiện (tr.m3),Tháng này - %KHCP,Tháng này - %KHQT,SL hiện tại - Cộng dồn (tr.m3),SL hiện tại - Cộng dồn (tr.ft3),SL hiện tại - %KHCP,SL hiện tại - %KHQT,SL ngày (tr.m3),SL ngày (tr.ft3),Dữ liệu cập nhật đến ngày
0,Bạch Hổ+ Rồng,90.00,122.00,61.38,68.20,50.31,9.56,13.28,0.46,4.82,3.47,61.84,2184.01,68.72,50.69,0.46,16.28,01/07/2025
1,Tê Giác Trắng,34.53,34.53,34.76,100.68,100.68,2.81,2.23,0.24,8.40,10.58,35.00,1236.01,101.36,101.36,0.24,8.34,01/07/2025
2,Rạng Đông+Phương Đông,34.35,100.00,38.34,111.64,38.34,0.00,0.00,0.13,0.00,0.00,38.47,1358.72,112.02,38.47,0.13,4.64,01/07/2025
3,Chim Sáo+ Dừa,113.83,117.30,38.41,33.74,32.74,9.63,10.91,0.23,2.34,2.06,38.63,1364.21,33.94,32.93,0.23,7.95,01/07/2025
4,STĐ+STV+STT+STN,405.20,441.19,153.32,37.84,34.75,34.41,41.89,0.28,0.82,0.68,153.60,5424.33,37.91,34.81,0.28,10.00,01/07/2025
5,Cá Ngừ Vàng,27.80,30.00,27.84,100.15,92.81,2.00,1.78,0.38,19.01,21.32,28.22,996.66,101.52,94.07,0.38,13.43,01/07/2025
6,Lan Tây+Lan Đỏ,0.00,30.81,60.63,0.00,196.79,0.00,0.00,0.01,0.00,0.00,60.65,2141.67,0.00,196.83,0.01,0.42,01/07/2025
7,Rồng Đôi+Rồng Đôi Tây,261.20,266.60,172.02,65.86,64.52,23.91,23.05,0.61,2.53,2.63,172.63,6096.28,66.09,64.75,0.61,21.40,01/07/2025
8,Lô PM3-CAA ( tổng khí về bờ),992.00,1935.42,928.76,93.62,47.99,85.14,168.80,5.24,6.15,3.10,933.99,32983.65,94.15,48.26,5.24,184.91,01/07/2025
9,Hải Sư Trắng +Hải Sư Đen,23.00,38.00,13.28,57.73,34.94,1.93,4.41,0.18,9.54,4.17,13.46,475.39,58.53,35.43,0.18,6.50,01/07/2025


### CREATE SQL

In [ ]:
# import pandas as pd

# field_df = pd.read_csv("data/formatted/csv/to_sql_fields.csv")
# plan_prod = pd.read_csv("data/formatted/csv/to_sql_planning_prod.csv")
# daily_prod = pd.read_csv("data/formatted/csv/to_sql_daily_prod.csv")

# # Convert report_date to datetime
# plan_prod['report_date'] = pd.to_datetime(plan_prod['report_date'], format='%d/%m/%Y', errors='coerce')
# daily_prod['report_date'] = pd.to_datetime(daily_prod['report_date'], format='%d/%m/%Y', errors='coerce')

In [ ]:
# PGDB.delete_table("daily_prod")
# PGDB.delete_table("plan_prod")
# PGDB.delete_table("field")

# PGDB.create_field_table()
# PGDB.create_plan_prod_table()
# PGDB.create_daily_prod_table()

In [ ]:
# for index, row in field_df.iterrows():
#     PGDB.insert_field(
#         field_id=row['short_name'],
#         field_name=row['full_name'],
#         unit=row['unit'],
#         location=row['location'],
#         field_type=row['prod_type'],
#         conversion_factor=row['conversion_factor']
#     )

# for index, row in plan_prod.iterrows():
#     PGDB.insert_plan_prod(
#         field_id=row['field_id'],
#         report_date=row['report_date'],
#         plan_type=row['plan_type'],
#         prod_ton=row['prod_ton'],
#         prod_bbls=row['prod_bbls'],
#         prod_m3=row['prod_m3'],
#         prod_ft3=row['prod_ft3']
#     )

# for index, row in daily_prod.iterrows():
#     PGDB.insert_daily_prod(
#         field_id=row['field_id'],
#         report_date=row['report_date'],
#         prod_type=row['prod_type'],
#         prod_ton=row['prod_ton'],
#         prod_bbls=row['prod_bbls'],
#         prod_m3=row['prod_m3'],
#         prod_ft3=row['prod_ft3']
#     )